In [1]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
user_data = pd.read_csv('Data/user_data.csv')
ads_data = pd.read_csv('Data/ads_data.csv')
user_messages = pd.read_csv('Data/user_messages.csv')
user_messages_test = pd.read_csv('Data/user_messages_test.csv')
ads_data = ads_data.loc[ads_data.enabled==1]
ads_data.reset_index(drop=True, inplace=True)

In [5]:
user_data.replace(to_replace={'event':{'view':0, 'first_message':1}}, inplace=True)
user_data.replace(to_replace={'origin':{'browse_search':0, 'search':0, 'browse':0, 'home':1,
                                       'notification_center':1, 'push':2, 'drawer':2, 'deeplink':1}}, inplace=True)
user_data.replace(to_replace={'channel': {'android': 2, 'ios':1, 'desktop':0}}, inplace=True )

In [4]:
user_data['event_time'] = pd.to_datetime(user_data.event_time)
user_data['event_month'] = user_data['event_time'].dt.month
user_data['event_day'] = user_data['event_time'].dt.day
user_data['hour'] = user_data['event_time'].dt.hour

In [5]:
from sklearn.preprocessing import LabelEncoder
le_user_id = LabelEncoder()
le_user_id = le_user_id.fit(list( (set(user_messages_test.user_id)).union(set(user_data.user_id)) ))
user_data['encoded_user_id'] = le_user_id.transform(user_data.user_id)

In [6]:
le_ad_id = LabelEncoder()
le_ad_id = le_ad_id.fit(list(set(user_data.ad_id).union(set(ads_data.ad_id))))
user_data['encoded_ad_id'] = le_ad_id.transform(user_data.ad_id)
ads_data['encoded_ad_id'] = le_ad_id.transform(ads_data.ad_id)

In [7]:
le_cat_id = LabelEncoder()
ads_data['encoded_category_id'] = le_cat_id.fit_transform(ads_data.category_id)

In [8]:
from scipy import sparse

In [9]:
s3 = set(pd.unique(user_data.user_id))
s4 = set(pd.unique(user_messages_test.user_id))
nu = len(s3.union(s4))
         
s1 = set(pd.unique(ads_data.ad_id))
s2 = set(pd.unique(user_data.ad_id))
na = len(s1.union(s2))
n = user_data.shape[0]
user_ad_mat = sparse.coo_matrix( ( np.ones(n), (user_data.encoded_user_id, user_data.encoded_ad_id) ) ,shape=[nu,na])
user_ad_mat = user_ad_mat.tocsr()

In [10]:
nc = len(pd.unique(ads_data.category_id))
n = ads_data.shape[0]
ad_cat_mat = sparse.coo_matrix( ( np.ones(n), ( ads_data.encoded_ad_id, ads_data.encoded_category_id )  ), shape=[na,nc] )
ad_cat_mat = ad_cat_mat.tocsr()

In [11]:
rem_ad_id = list(set(user_data.ad_id) - set(ads_data.ad_id))
rem_cat_id = list(np.zeros(len(rem_ad_id)))

In [12]:
ad_cat_df = pd.DataFrame(data=np.array(list(ads_data.category_id)+rem_cat_id), index=list(ads_data.ad_id)+rem_ad_id)

In [13]:
ad_info_df = pd.DataFrame(data=np.array(user_data), index=user_data.ad_id, columns=user_data.columns)

In [14]:
user_user_mat = user_ad_mat * user_ad_mat.transpose()

In [15]:
lst = []
for u,c in user_messages_test.values:
    user = user_user_mat[le_user_id.transform([u])[0],:]
    a = np.sum(user_ad_mat.transpose().multiply(user), axis=1)
    a = np.array(a.reshape([1,-1]))[0]
    b = le_ad_id.inverse_transform(np.argsort(-a))
    cat_b = ad_cat_df.loc[b]
    cat_b = np.array(cat_b).reshape([b.shape[0]])
    b = b[cat_b==c]
    lst.append(str(list(b[:10])))
    cnt+=1

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500


In [16]:
ad_id = pd.Series(lst)
submission = user_messages_test.copy()
submission['ads'] = ad_id

In [18]:
submission.to_csv('sub_mat.csv', index=False)